Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Loading the data

In [ ]:
df = pd.read_csv('/content/movies.csv', on_bad_lines='skip')
print("CSV read successfully by skipping problematic lines.")

CSV read successfully by skipping problematic lines.


In [ ]:
pd.set_option('display.max_columns', None)

Viewing the top 5 rows

In [ ]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
df.shape

(4803, 24)

In [ ]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

#index: A unique identifier for each row in the dataset.
#budget: The estimated production cost of the movie in dollars.
#genres:A list of genres the movie belongs to (e.g., Action, Drama, Comedy).
#homepage: The official website URL of the movie.
#id : A unique identifier assigned to the movie, typically from a database like TMDB or IMDb.
#keywords: A list of keywords describing the movie (e.g., "superhero," "time travel").
#original_language: The primary language in which the movie was originally produced.
#original_title :The original title of the movie before translation/localization.
#overview : A short summary of the movie's plot.
#popularity: A numerical score representing the movie's popularity, often derived from searches, views, and social media interactions.
#production_companies : A list of companies that produced the movie.
#production_countries :The countries where the movie was produced.
#release_date : The date when the movie was released in theaters.
#revenue : The total earnings of the movie, usually in dollars.
#runtime:The duration of the movie in minutes.
#spoken_languages: The languages spoken in the movie.
#status: The current status of the movie (e.g., "Released," "Post Production").
#tagline: A short promotional phrase or slogan associated with the movie.
#title: The official title of the movie.
#vote_average: The average rating given by users.
#vote_count:The total number of votes or ratings received.
#cast: The list of main actors/actresses in the movie.
#crew : The list of crew members involved in production, such as writers, producers, and cinematographers.
#director: The name of the movie's director.

Data Preprocessing

In [ ]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
# replacing the null valuess with null string

for feature in selected_features:
  df[feature] = df[feature].fillna('')

In [ ]:
# combining all the 5 selected features

combined_features = df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [ ]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.24713765026964
  (4801, 17266)	0.28860981849329476
  (4801, 13835)	0.27870029291200094
  (4801, 13175)	0.28860981849329476
  (4801, 171

Cosine similarity

In [ ]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [ ]:
print(similarity.shape)

(4803, 4803)


Getting the movie name from the user

In [ ]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : Titanic


In [ ]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = df['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Titanic', 'Trance', 'Maniac']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Titanic


In [ ]:
# finding the index of the movie with title

index_of_the_movie = df[df.title == close_match]['index'].values[0]
print(index_of_the_movie)

25


In [ ]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Titanic', 'Trance', 'Maniac']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Titanic


In [ ]:
# finding the index of the movie with title

index_of_the_movie = df[df.title == close_match]['index'].values[0]
print(index_of_the_movie)

25


In [ ]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.04284926095950226), (1, 0.0), (2, 0.011467364105495698), (3, 0.010046067299207016), (4, 0.010913900929666012), (5, 0.013271868195005247), (6, 0.0), (7, 0.04427635120584794), (8, 0.0), (9, 0.01105296767401108), (10, 0.04077473745704553), (11, 0.005154880168301485), (12, 0.0), (13, 0.011810642378201053), (14, 0.011135108727728623), (15, 0.010700980101090706), (16, 0.010727286350311041), (17, 0.0), (18, 0.0), (19, 0.058321349267840134), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0030136767341105873), (25, 1.0000000000000004), (26, 0.0), (27, 0.035917907722692974), (28, 0.005803448123074126), (29, 0.017798015596744543), (30, 0.01291221720585476), (31, 0.01101477659790138), (32, 0.010369537913084387), (33, 0.017063631264570205), (34, 0.02704944297060011), (35, 0.0), (36, 0.0), (37, 0.04995428411323933), (38, 0.011438736022572782), (39, 0.0), (40, 0.0), (41, 0.005589882450477924), (42, 0.0), (43, 0.005898442690409995), (44, 0.013809733743104851), (45, 0.01978396131348192), (46

In [ ]:
len(similarity_score)

4803

In [ ]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(25, 1.0000000000000004), (593, 0.2300727107211525), (454, 0.16589907730438733), (1081, 0.16035128823311431), (2011, 0.13803443166572782), (3631, 0.1333839815786052), (4133, 0.13183140853614747), (656, 0.12606246128803086), (1985, 0.12349759621747017), (2036, 0.12348320550112096), (316, 0.11879207685844573), (142, 0.11838847957751894), (2449, 0.11814094328522082), (846, 0.11610782635112268), (49, 0.11503079796465221), (765, 0.1136042644493443), (439, 0.11317975752102441), (2245, 0.11276113457467152), (720, 0.10752741389011594), (1362, 0.10696934437645386), (1629, 0.10631964670577465), (1500, 0.10616709183973469), (972, 0.10565080846258226), (622, 0.10374165072398074), (2955, 0.10279003651035282), (2175, 0.1024814836191804), (250, 0.1007017696672862), (4231, 0.09751550112290147), (1089, 0.09711335392784966), (2008, 0.09668793526881608), (961, 0.09638190705612418), (351, 0.09601987660900405), (3533, 0.09505757385996452), (249, 0.09498018720867245), (176, 0.09472769132005916), (2152, 0.0

In [ ]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index =df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Titanic
2 . The Dilemma
3 . The Day the Earth Stood Still
4 . Revolutionary Road
5 . Cheri
6 . Heavenly Creatures
7 . Plush
8 . Primary Colors
9 . The Bridge of San Luis Rey
10 . Saving Silverman
11 . Gangs of New York
12 . Flushed Away
13 . Sense and Sensibility
14 . Stuck on You
15 . The Great Gatsby
16 . Almost Famous
17 . Shutter Island
18 . The Brothers Bloom
19 . Contagion
20 . The Blind Side
21 . Little Black Book
22 . This Is the End
23 . The Host
24 . Body of Lies
25 . Romance & Cigarettes
26 . Ladyhawke
27 . The Aviator
28 . The Believer
29 . The Phantom


Movie Recommendation Sytem

In [ ]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie =df[df.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : joe
Movies suggested for you : 

1 . Joe
2 . Mud
3 . Gory Gory Hallelujah
4 . All the Real Girls
5 . Mystic River
6 . Leaving Las Vegas
7 . Kiss of Death
8 . Astro Boy
9 . Sleepers
10 . Oliver Twist
11 . The Ant Bully
12 . The Celebration
13 . The Sixth Sense
14 . George Washington
15 . Shine a Light
16 . Volver
17 . Bringing Out the Dead
18 . 8MM
19 . Your Highness
20 . The Sitter
21 . Radio Flyer
22 . Snow Angels
23 . Drive Angry
24 . Killing Zoe
25 . August Rush
26 . Dying of the Light
27 . The Sorcerer's Apprentice
28 . Pineapple Express
29 . I Think I Love My Wife
